In [22]:
import os
import librosa

import pandas as pd

In [2]:
from utils.misc import recursive_search

In [3]:
DATA_PATH = os.path.join(os.getcwd(), "data", "voxforge")
audio_converted_path = os.path.join(DATA_PATH, "audio_converted")

In [4]:
audio_converted_path

'/home/dima/Projects/stt_uk/data/voxforge/audio_converted'

## Audio data

In [5]:
audio_df = recursive_search(audio_converted_path)
audio_df['duration'] = audio_df['file_path'].map(lambda x: librosa.get_duration(filename=x))
audio_df['file_size'] = audio_df['file_path'].map(lambda x: os.path.getsize(x))

In [8]:
print(audio_df.shape)
print(f"Total duration: {audio_df['duration'].sum()/60/60:.2f} hours")
print(f"Total size: {audio_df['file_size'].sum()/1024/1024:.2f} mb")
audio_df.sample(5)

(50, 4)
Total duration: 0.13 hours
Total size: 6.90 mb


,file_name,file_path,duration,file_size
38,Darrr_uk_0050.wav,/home/dima/Projects/stt_uk/data/voxforge/audio...,5.875,94044
49,Darrr_uk_0049.wav,/home/dima/Projects/stt_uk/data/voxforge/audio...,9.500,152044
7,Alias_uk_0025.wav,/home/dima/Projects/stt_uk/data/voxforge/audio...,7.125,114044
37,Alias_uk_0028.wav,/home/dima/Projects/stt_uk/data/voxforge/audio...,8.750,140044
21,Alias_uk_0023.wav,/home/dima/Projects/stt_uk/data/voxforge/audio...,5.250,84044


## text and gender

In [17]:
text_df = pd.read_csv(os.path.join(DATA_PATH, "voxforge_transcript.csv"))
text_df['speaker_id'] = text_df.file_name.map(lambda x: x.split("_", 1)[0])

In [21]:
print(text_df.shape)
text_df.head()

(50, 5)


,file_name,transcript,gender,speaker_id,transcript_clean
0,Neverice_uk_0031.wav,"Водночас у тексті є багато думок, на які хочет...",m,Neverice,водночас у тексті є багато думок на які хочеть...
1,Neverice_uk_0032.wav,"Тяжкою працею, з допомогою щоразу більшої груп...",m,Neverice,тяжкою працею з допомогою щоразу більшої групи...
2,Neverice_uk_0033.wav,"Досі не бралася за цю тему, вважаючи, що це зр...",m,Neverice,досі не бралася за цю тему вважаючи що це зроб...
3,Neverice_uk_0034.wav,На початку його каденції ці рішення не були по...,m,Neverice,на початку його каденції ці рішення не були по...
4,Neverice_uk_0035.wav,"Розподіл української єпархії між галичанами, я...",m,Neverice,розподіл української єпархії між галичанами як...


### Prepare text

In [28]:
import re

from tqdm import tqdm
from collections import Counter

In [29]:
def regex_token(x):
    return " ".join(re.findall(r"\w+", x)).lower()

In [30]:
text_df['transcript_clean'] = text_df.transcript.map(regex_token)

In [31]:
lst = []

for item in tqdm(text_df["transcript_clean"].str.split().values):
    lst.extend(item)

100%|██████████| 50/50 [00:00<00:00, 175493.89it/s]


In [32]:
Counter(lst).most_common(10)

[('не', 32),
 ('і', 27),
 ('що', 21),
 ('на', 12),
 ('та', 10),
 ('те', 8),
 ('щось', 8),
 ('все', 8),
 ('є', 7),
 ('з', 7)]

### Kaldi requirements

#### corpus.txt

In [ ]:
df.loc[df["transcript"].str.len() > 1]["transcript"].unique()